In [ ]:
import sys
sys.path.append("..")

import sympy
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter

import tensorflow as tf
import tensorflow_quantum as tfq
import cirq

from cv_ops import PositionOp, MomentumOp
from cv_subroutines import centeredQFT

In [ ]:
def U_mix(qubits, gamma):
    d = 2 ** len(qubits)
    return centeredQFT(qubits, inverse=True) + tfq.util.exponential(operators = [(1/2) * MomentumOp(qubits).op ** 2], coefficients = gamma) + centeredQFT(qubits)

def H_cost_poly(qubits, poly):
    H_c = cirq.PauliSum()
    for q in qubits:
        H_c += cirq.PauliString((poly[0]/len(qubits))*cirq.I(q))
    for idx, p in enumerate(poly[1:]):
        temp_op = p * PositionOp(qubits).op
        for _ in range(idx):
            temp_op *= PositionOp(qubits).op
        H_c += temp_op

    return H_c

In [ ]:
def generate_parameters(p):
    param_list = ['a','b']
    for _ in range(p - 1):
        param_list += [chr(ord(param_list[-1]) + 1), chr(ord(param_list[-1]) + 2)]
    param_string = param_list[0]
    for param in param_list[1:]:
        param_string += ' ' + param
    return sympy.symbols(param_string)

def qaoa_circuit(qubits, poly, p, qaoa_parameters):
    circuit = cirq.Circuit()
    for idx in range(p):
        circuit += tfq.util.exponential(operators = [H_cost_poly(qubits, poly)], coefficients = qaoa_parameters[2*idx:2*idx+1])
        circuit += U_mix(qubits, qaoa_parameters[2*idx+1:2*idx+2])
    return circuit

def hadamard_circuit(qubits):
    return cirq.Circuit([cirq.H(q) for q in qubits])

In [ ]:
def find_optimal_qaoa_params(qaoa_model, input_circs, optimizer, tol):
    old_func_val = np.inf
    vals = []
    diff = np.inf
    iter_num = 0
    while (diff > tol):
        if (iter_num + 1) % 100 == 0:
            print("Iteration " + str(iter_num + 1) + "\nFunction Value: " + str(vals[-1]))
        with tf.GradientTape() as tape:
            func_val = qaoa_model(tfq.convert_to_tensor(input_circs))
        gradients = tape.gradient(func_val, qaoa_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, qaoa_model.trainable_variables))
        func_val = func_val.numpy()[0][0]
        vals.append(func_val)
        diff = abs(func_val - old_func_val)
        old_func_val = func_val
        iter_num += 1
    return vals

In [ ]:
def run_qaoa():
    poly_to_optimize = [10.0, -5.0, 1.0]
    n_qubits = 4
    qubits = [cirq.GridQubit(0,i) for i in range(n_qubits)]
    p = 10
    qaoa_parameters = generate_parameters(p)

    learning_rate = 0.01
    optimizer  = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    tol = 1e-7
    
    input_circs = [hadamard_circuit(qubits)]
    qaoa_circ = qaoa_circuit(qubits, poly_to_optimize, p, qaoa_parameters)
    model_readout = H_cost_poly(qubits, poly_to_optimize)
    inputs = tf.keras.layers.Input(shape=(), dtype=tf.dtypes.string)
    qaoa_pqc = tfq.layers.PQC(qaoa_circ, model_readout, differentiator=tfq.differentiators.Adjoint())(inputs)
    qaoa_model = tf.keras.models.Model(inputs=inputs, outputs=qaoa_pqc)

    func_val_history = find_optimal_qaoa_params(qaoa_model, input_circs, optimizer, tol)

    # Read out the optimal paramters and sample from the final state 1000 times
    params = qaoa_model.trainable_variables
    add = tfq.layers.AddCircuit()
    output_circuit = add(tfq.convert_to_tensor(input_circs), append = qaoa_circuit(qubits, poly_to_optimize, p, qaoa_parameters))

    sample_layer = tfq.layers.Sample()
    output = sample_layer(output_circuit,  symbol_names=qaoa_parameters, symbol_values = params, repetitions=1000)

    return output

def plot_output(output):
    count = Counter(["".join([str(bit) for bit in bitstring]) for bitstring in output.numpy()[0]])
    cirq.plot_state_histogram(count, plt.subplot())
    plt.show()
    # plt.savefig("output_histogram.png")

In [ ]:
output = run_qaoa()
plot_output(output)